In [1]:
from main import query_prices_and_save_to_db, today_tomorrow_str_dates
import pandas as pd

In [2]:
# INPUTS
yesterday_str, tomorrow_str = today_tomorrow_str_dates()
start = pd.Timestamp(yesterday_str, tz='Europe/Brussels')
end = pd.Timestamp(tomorrow_str, tz='Europe/Brussels')
country_code = 'ES'
start = pd.Timestamp('20220101', tz='Europe/Brussels')
end = pd.Timestamp('20221231', tz='Europe/Brussels')

In [3]:
query_prices_and_save_to_db(start, end, country_code, save_to_path = False)

(pymysql.err.OperationalError) (1170, "BLOB/TEXT column 'unique_id' used in key specification without a key length")
[SQL: ALTER TABLE `prices-table` ADD PRIMARY KEY (`unique_id`);]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
